**Notebook for 5-anonymizing a redacted version of the EdX dataset.**

**Instructions**: Assuming that each field other than the user-id and the course name is a quasi-identifier, determine the level of k-anonymity in the file. Then, make the file 5-anonymous using only record suppression; how many records need to be deleted to do this? Try making the file 5-anonymous using only column suppression; how many columns need to be deleted to do this, and which ones are they? Finally, try to produce a 5-anonymous data set using generalization. Finally, see if you can use some combination of these mechanisms to produce a 5-anonymous data set.

In [9]:
# import the dataset as a pandas dataframe
import pandas as pd
import numpy as np

df_init = pd.read_csv("105facebook.csv")
df_init

,Name,Email,House,Year,Concentration
0,Aishani Aatresh,aishaniaatresh@college.harvard.edu,Leverett,Senior,Special Concentrations
1,Isaiah Abbey,NaN,Eliot,Sophomore,Undeclared
2,Kian Abbott,kianabbott@college.harvard.edu,Pforzheimer,Senior,Electrical Engineering
3,Julia Abbruzzese,juliaabbruzzese@college.harvard.edu,Quincy,Senior,Psychology
4,Omar Abdel Haq,omar_abdelhaq@college.harvard.edu,Winthrop,Senior,Applied Mathematics
...,...,...,...,...,...
5771,Grace von Oiste,NaN,Quincy,Senior,Statistics
5772,Henry von der Schulenburg,NaN,Pforzheimer,Senior,Applied Mathematics
5773,Onyx ¿wa,eksmith@college.harvard.edu,Winthrop,Senior,"Art, Film, and Visual Studies"
5774,Ashley Álvarez,aalvarez@college.harvard.edu,Lowell,Senior,Sociology


In [10]:
# drop last row since NaN
df = df_init.iloc[:-1 , :]
df

,Name,Email,House,Year,Concentration
0,Aishani Aatresh,aishaniaatresh@college.harvard.edu,Leverett,Senior,Special Concentrations
1,Isaiah Abbey,NaN,Eliot,Sophomore,Undeclared
2,Kian Abbott,kianabbott@college.harvard.edu,Pforzheimer,Senior,Electrical Engineering
3,Julia Abbruzzese,juliaabbruzzese@college.harvard.edu,Quincy,Senior,Psychology
4,Omar Abdel Haq,omar_abdelhaq@college.harvard.edu,Winthrop,Senior,Applied Mathematics
...,...,...,...,...,...
5770,Hannah van der Sluijs,hvandersluijs@college.harvard.edu,Lowell,Sophomore,Undeclared
5771,Grace von Oiste,NaN,Quincy,Senior,Statistics
5772,Henry von der Schulenburg,NaN,Pforzheimer,Senior,Applied Mathematics
5773,Onyx ¿wa,eksmith@college.harvard.edu,Winthrop,Senior,"Art, Film, and Visual Studies"


In [35]:
def get_quasi_ids(df):
    """
    Finds the quasi identifiers in the given dataframe,
    which is all column names, except for 'Email' and 'Name'
    """
    quasi_ids = df.columns.to_list()
    quasi_ids.remove("Email")
    quasi_ids.remove("Name")
    return quasi_ids

In [36]:
# get quasi_ids
quasi_ids = get_quasi_ids(df)
quasi_ids

['House', 'Year', 'Concentration']

In [37]:
# Function for determining level of k-anonymity in a file
# 
def level_k_anon(df, quasi_ids=quasi_ids):
    """
    Determines the level of k anonymity the given dataframe has
    for the given list of quasi identifier names.

    Parameters:
    -----------
    df: pandas DataFrame
        df to find the level of anonymity of

    quasi_ids: list
        list of names of quasi identifiers (col names that correspond
        to quasi identifiers)
        NOTE: default is all column names except "course_id" and "user_id"
    """
    # Group by set of quasi id values
    quasi_id_grouped_df = df.groupby(quasi_ids, dropna=False)
    # Get number of rows in each gruop
    grouped_row_counts = quasi_id_grouped_df.size()
    # Min number of rows in a group = level of k-anonymity
    level_k_anon_num = min(grouped_row_counts)
    return level_k_anon_num
    
    

**1. Determine the level of k-anonymity in the file**

In [40]:
original_level_k_anon = level_k_anon(df)
original_level_k_anon

1

1. **Result:** the datframe is currently 1-anonymous

**2. Make file anonymous using record suppression.**  (Assuming records are rows)

**Question:** how many records need to be deleted to do this?

In [54]:
k = 5

In [55]:
num_records_dropped = 0
# Group the dataframe by each unique set of quasi id values
df_grouped_by_quasi_ids = df.groupby(quasi_ids, dropna=False)
# Iterate through each group, removing any with < 5 entries
for name, group in df_grouped_by_quasi_ids:
    # print(name)
    # print(group)
    # Get size of group
    group_size = group.shape[0]
    # print(f"num of rows in group: {group_size}")
    if group_size < k:
        # print("group size < 0")
        # Record how many entries were removed when the group was removed
        num_records_dropped += group_size
    # print("\n")

print(f"total num of records dropped: {num_records_dropped}")

total num of records dropped: 2735


In [56]:
# Verify results
# Group by all the quasi ids
group_by_quasi_ids_df = df.groupby(quasi_ids, dropna=False)
# Get number of rows in each gruop
grouped_row_counts = group_by_quasi_ids_df.size()
print(grouped_row_counts)

# Get number of rows who belong to a group of quasi ids with less than k members
print(f"k: {k}")
num_rows_less_than_k = grouped_row_counts[grouped_row_counts < k ].sum(skipna=False) 
print(f"num rows with less than k duplicates: {num_rows_less_than_k}")
num_rows_greater_than_k = grouped_row_counts[grouped_row_counts >= k ].sum(skipna=False) 
print(f"num rows with > or = to k duplicates: {num_rows_greater_than_k}")

House     Year       Concentration                                        
Adams     Junior     Anthropology; Human Evolutionary Biology                 1
                     Applied Mathematics                                      9
                     Art, Film, and Visual Studies                            1
                     Art, Film, and Visual Studies; Comparative Literature    1
                     Art, Film, and Visual Studies; History                   1
                                                                             ..
Winthrop  Sophomore  Sociology; Psychology                                    1
                     Statistics                                               3
                     Statistics; Economics                                    1
                     Statistics; Psychology                                   1
                     Undeclared                                               6
Length: 2156, dtype: int64
k: 5
num rows with

2. **Result**: 2735 records were deleted to make the dataset 5-anonymous using record supression.

**3. Make the file 5-anonymous using only column suppression**

**Question:** How many columns are needed to do this, and which ones are they? 

In [64]:
# Step 1: group by each col, throw out cols that results in groups with < k rows
cols_removed = []
for quasi_id in quasi_ids:
    # Isolate just the column for a particular quasi id
    col_df = df[quasi_id]
    # Count the number of occurrences (counts) of each unique value in it
    count = df[quasi_id].value_counts()
    # print(count)
    # Get the minimum number of occurrences of a unique value (min count at end)... 
    # if this is < 5, this record must be dropped
    min_count = count.iloc[-1]
    print(f"col for {quasi_id} has a min value count of {min_count}")
    if (min_count < k):
        cols_removed.append(quasi_id)

print(f"cols removed: {cols_removed}")
print(f"num cols removed: {len(cols_removed)}")

col for House has a min value count of 123
col for Year has a min value count of 49
col for Concentration has a min value count of 1
cols removed: ['Concentration']
num cols removed: 1


In [63]:
# Step 2: check if reached 5-anonymity
init_cols_supressed_df = df.drop(cols_removed,axis=1)
# Checking that dropped all 11 desired cols
print(f"init cols supressed df: \n{init_cols_supressed_df}")
# Check level of anonymity in remaining df
sup_quasi_ids = get_quasi_ids(init_cols_supressed_df)
print(f"Quasi ids of col supressed df: {sup_quasi_ids}")
init_col_sup_lvl_anon = level_k_anon(init_cols_supressed_df, quasi_ids=sup_quasi_ids)
print(f"Level of k anonymity of dataframe after step 1: {init_col_sup_lvl_anon}")

init cols supressed df: 
                           Name                                Email  \
0               Aishani Aatresh   aishaniaatresh@college.harvard.edu   
1                  Isaiah Abbey                                  NaN   
2                   Kian Abbott       kianabbott@college.harvard.edu   
3              Julia Abbruzzese  juliaabbruzzese@college.harvard.edu   
4                Omar Abdel Haq    omar_abdelhaq@college.harvard.edu   
...                         ...                                  ...   
5770      Hannah van der Sluijs    hvandersluijs@college.harvard.edu   
5771            Grace von Oiste                                  NaN   
5772  Henry von der Schulenburg                                  NaN   
5773                   Onyx ¿wa          eksmith@college.harvard.edu   
5774             Ashley Álvarez         aalvarez@college.harvard.edu   

            House       Year  
0        Leverett     Senior  
1           Eliot  Sophomore  
2     Pforzheimer

3. **Result:** 1 column has to be dropped.  This is: 'Concentration'.  The level of k-anonymity after dropping this column is 23

In [66]:
# Exploratory: what is the level of anonymity if we drop the column with the 2nd-smallest level of duplicates as well?
explor_cols_supressed_df = df.drop(cols_removed + ["House"],axis=1)
# Checking that dropped all 11 desired cols
print(f"explor cols supressed df: \n{explor_cols_supressed_df}")
# Check level of anonymity in remaining df
sup_quasi_ids2 = get_quasi_ids(explor_cols_supressed_df)
print(f"Quasi ids of col supressed df: {sup_quasi_ids2}")
explor_col_sup_lvl_anon = level_k_anon(explor_cols_supressed_df, quasi_ids=sup_quasi_ids2)
print(f"Level of k anonymity of dataframe after exploring deleting 'Concentration' and 'House': {explor_col_sup_lvl_anon}")

explor cols supressed df: 
                           Name                                Email  \
0               Aishani Aatresh   aishaniaatresh@college.harvard.edu   
1                  Isaiah Abbey                                  NaN   
2                   Kian Abbott       kianabbott@college.harvard.edu   
3              Julia Abbruzzese  juliaabbruzzese@college.harvard.edu   
4                Omar Abdel Haq    omar_abdelhaq@college.harvard.edu   
...                         ...                                  ...   
5770      Hannah van der Sluijs    hvandersluijs@college.harvard.edu   
5771            Grace von Oiste                                  NaN   
5772  Henry von der Schulenburg                                  NaN   
5773                   Onyx ¿wa          eksmith@college.harvard.edu   
5774             Ashley Álvarez         aalvarez@college.harvard.edu   

           Year  
0        Senior  
1     Sophomore  
2        Senior  
3        Senior  
4        Senior  


**4. Produce a 5-anonymous dataset using generalization**

4.1: Step 1: generalize the 1 column, "Concentration", that did not have at least 5 entries per unique value

In [67]:
def get_grouped_row_counts(df, cols):
    grouped_df = df.groupby(cols, dropna=False)
    return grouped_df.size()

# Declare df for generalization to occur in
gen_df = df.copy()
gen_df

,Name,Email,House,Year,Concentration
0,Aishani Aatresh,aishaniaatresh@college.harvard.edu,Leverett,Senior,Special Concentrations
1,Isaiah Abbey,NaN,Eliot,Sophomore,Undeclared
2,Kian Abbott,kianabbott@college.harvard.edu,Pforzheimer,Senior,Electrical Engineering
3,Julia Abbruzzese,juliaabbruzzese@college.harvard.edu,Quincy,Senior,Psychology
4,Omar Abdel Haq,omar_abdelhaq@college.harvard.edu,Winthrop,Senior,Applied Mathematics
...,...,...,...,...,...
5770,Hannah van der Sluijs,hvandersluijs@college.harvard.edu,Lowell,Sophomore,Undeclared
5771,Grace von Oiste,NaN,Quincy,Senior,Statistics
5772,Henry von der Schulenburg,NaN,Pforzheimer,Senior,Applied Mathematics
5773,Onyx ¿wa,eksmith@college.harvard.edu,Winthrop,Senior,"Art, Film, and Visual Studies"


4.1.1 generalize "Concentration"

In [219]:
cc_by_ip_counts_dict = get_grouped_row_counts(df, ["cc_by_ip"]).to_dict()
print(cc_by_ip_counts_dict)
# Easiest way is to probably group things into regions
# Got a csv mapping codes to subregions and regions, doing subregions first
# to try to preserve info
country_code_df = pd.read_csv("country_code_region.csv")
country_code_df

{'AD': 3, 'AE': 718, 'AF': 51, 'AG': 16, 'AI': 1, 'AL': 194, 'AM': 116, 'AO': 33, 'AR': 1086, 'AS': 1, 'AT': 535, 'AU': 3703, 'AW': 10, 'AX': 2, 'AZ': 99, 'BA': 113, 'BB': 47, 'BD': 569, 'BE': 963, 'BF': 32, 'BG': 353, 'BH': 85, 'BI': 10, 'BJ': 24, 'BM': 13, 'BN': 26, 'BO': 169, 'BQ': 2, 'BR': 6615, 'BS': 49, 'BT': 25, 'BW': 59, 'BY': 152, 'BZ': 27, 'CA': 6475, 'CD': 43, 'CF': 1, 'CG': 4, 'CH': 833, 'CI': 74, 'CL': 952, 'CM': 177, 'CN': 3510, 'CO': 2330, 'CR': 304, 'CU': 24, 'CV': 19, 'CW': 15, 'CY': 93, 'CZ': 474, 'DE': 3425, 'DJ': 24, 'DK': 505, 'DM': 11, 'DO': 272, 'DZ': 316, 'EC': 498, 'EE': 138, 'EG': 1629, 'ER': 6, 'ES': 3220, 'ET': 282, 'FI': 363, 'FJ': 34, 'FM': 3, 'FO': 4, 'FR': 2801, 'GA': 8, 'GB': 7638, 'GD': 18, 'GE': 182, 'GF': 1, 'GG': 4, 'GH': 597, 'GI': 3, 'GL': 4, 'GM': 25, 'GN': 15, 'GP': 7, 'GQ': 2, 'GR': 1621, 'GT': 203, 'GU': 17, 'GW': 1, 'GY': 40, 'HK': 1378, 'HN': 127, 'HR': 285, 'HT': 138, 'HU': 448, 'ID': 1296, 'IE': 746, 'IL': 575, 'IM': 5, 'IN': 14752, 'IQ': 

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,NaN,9.0,61.0,NaN
245,Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,NaN,2.0,15.0,NaN
246,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,NaN,142.0,145.0,NaN
247,Zambia,ZM,ZMB,894,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0


In [223]:
# Map each cc_by_ip = alpha-2 to sub-region-code through this csv
# Create mappings between cc and sub-region
mappings = {}
for cc, count in cc_by_ip_counts_dict.items():
    try:
        # TODO may have to change to region instead
        sub_region = country_code_df[country_code_df['alpha-2'] == cc]['sub-region'].values[0]
        mappings[cc] = sub_region
    except:
        # hit end of series
        pass
print(mappings)
gen_df["gen_cc_by_ip_to_sub-region"] = gen_df["cc_by_ip"].map(mappings)
gen_df = gen_df.drop(["cc_by_ip"],axis=1)

{'AD': 'Southern Europe', 'AE': 'Western Asia', 'AF': 'Southern Asia', 'AG': 'Latin America and the Caribbean', 'AI': 'Latin America and the Caribbean', 'AL': 'Southern Europe', 'AM': 'Western Asia', 'AO': 'Sub-Saharan Africa', 'AR': 'Latin America and the Caribbean', 'AS': 'Polynesia', 'AT': 'Western Europe', 'AU': 'Australia and New Zealand', 'AW': 'Latin America and the Caribbean', 'AX': 'Northern Europe', 'AZ': 'Western Asia', 'BA': 'Southern Europe', 'BB': 'Latin America and the Caribbean', 'BD': 'Southern Asia', 'BE': 'Western Europe', 'BF': 'Sub-Saharan Africa', 'BG': 'Eastern Europe', 'BH': 'Western Asia', 'BI': 'Sub-Saharan Africa', 'BJ': 'Sub-Saharan Africa', 'BM': 'Northern America', 'BN': 'South-eastern Asia', 'BO': 'Latin America and the Caribbean', 'BQ': 'Latin America and the Caribbean', 'BR': 'Latin America and the Caribbean', 'BS': 'Latin America and the Caribbean', 'BT': 'Southern Asia', 'BW': 'Sub-Saharan Africa', 'BY': 'Eastern Europe', 'BZ': 'Latin America and the 

In [224]:
gen_df

,course_id,user_id,city,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,gen_cc_by_ip_to_sub-region
0,HarvardX/PH525.1x/1T2018,29940,Austin,78713,NaN,NaN,NaN,0,0,0,0,0,0,0,Northern America
1,HarvardX/PH525.1x/1T2018,37095,Dhaka,NaN,b,1991.0,m,0,0,0,0,0,0,0,Southern Asia
2,HarvardX/PH525.1x/1T2018,45634,Medellín,NaN,m,1982.0,m,0,0,0,0,0,0,0,Latin America and the Caribbean
3,HarvardX/PH525.1x/1T2018,52234,Skanör,NaN,p,1988.0,m,0,0,0,0,0,0,0,Northern Europe
4,HarvardX/PH525.1x/1T2018,52238,León,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,Latin America and the Caribbean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,HarvardX/Hum3.1x/1T2016,15291085,Silverdale,2752,jhs,2002.0,NaN,0,0,0,0,0,0,0,Australia and New Zealand
199995,HarvardX/Hum3.1x/1T2016,15292716,Yekaterinburg,620000,NaN,NaN,NaN,0,0,0,0,0,0,0,Eastern Europe
199996,HarvardX/Hum3.1x/1T2016,15295130,Istanbul,NaN,b,1996.0,f,0,0,0,0,0,0,0,Western Asia
199997,HarvardX/Hum3.1x/1T2016,15296396,Marshfield,02050,NaN,2000.0,NaN,1,0,0,0,1,0,0,Northern America


In [226]:
# Validate 5 anon of this col
gen_lvl_anon = level_k_anon(gen_df, ["gen_cc_by_ip_to_sub-region"])
gen_lvl_anon # Anon is 8, this is properly generalized!

8

4.1.2 Exploratory: generalize "House"

In [229]:
cities_counts_dict = get_grouped_row_counts(df, ["city"]).to_dict()
print(cities_counts_dict)
# Easiest way is to probably group cities into countries (by country codes)... if that fails:
# do sub-regions
# and if that fails: go back and do regions
# Got a csv mapping cities to countries
# to try to preserve info
cities_df = pd.read_csv("cities.csv")
cities_df

{"'s-hertogenbosch": 3, "'t Horntje": 1, 'A Coruña': 21, 'Aabenraa': 1, 'Aach': 1, 'Aachen': 30, 'Aalsmeer': 4, 'Aalst': 5, 'Aarburg': 1, 'Aarhus': 36, 'Aartselaar': 1, 'Aas': 3, 'Abakaliki': 1, 'Abakan': 2, 'Abancay': 2, 'Abano Terme': 2, 'Abarán': 1, 'Abasolo': 1, 'Abbekerk': 1, 'Abbeville': 3, 'Abbots Langley': 2, 'Abbotsford': 16, 'Abbottabad': 7, 'Abdullah': 1, 'Abeokuta': 5, 'Aberdare': 4, 'Aberdeen': 49, 'Abernethy': 1, 'Aberystwyth': 4, 'Abha': 1, 'Abidjan': 65, 'Abiko': 4, 'Abilene': 14, 'Abingdon': 13, 'Abira': 1, 'Abrams': 1, 'Absam': 1, 'Absecon': 59, 'Abu Dhabi': 174, 'Abuja': 176, 'Abymes': 1, 'Acacia Ridge': 1, 'Acacías': 2, 'Acapulco': 10, 'Acate': 1, 'Accra': 438, 'Accrington': 2, 'Acerra': 1, 'Acheng': 1, 'Achrafieh': 1, 'Acme': 1, 'Acolman': 1, 'Acton': 30, 'Actopan': 2, 'Acushnet': 1, 'Acworth': 15, 'Ad Dammam': 1, 'Ada': 4, 'Adamantina': 1, 'Adams': 2, 'Adana': 27, 'Addis Ababa': 178, 'Addison': 14, 'Addlestone': 3, 'Adeje': 1, 'Adelaide': 160, 'Adelsheim': 1, 'Adl

,id,name,state_id,state_code,state_name,country_id,country_code,country_name,latitude,longitude,wikiDataId
0,52,Ashkāsham,3901,BDS,Badakhshan,1,AF,Afghanistan,36.68333,71.53333,Q4805192
1,68,Fayzabad,3901,BDS,Badakhshan,1,AF,Afghanistan,37.11664,70.58002,Q156558
2,78,Jurm,3901,BDS,Badakhshan,1,AF,Afghanistan,36.86477,70.83421,Q10308323
3,84,Khandūd,3901,BDS,Badakhshan,1,AF,Afghanistan,36.95127,72.31800,Q3290334
4,115,Rāghistān,3901,BDS,Badakhshan,1,AF,Afghanistan,37.66079,70.67346,Q2670909
...,...,...,...,...,...,...,...,...,...,...,...
150548,131496,Redcliff,1957,MI,Midlands Province,247,ZW,Zimbabwe,-19.03333,29.78333,Q584001
150549,131502,Shangani,1957,MI,Midlands Province,247,ZW,Zimbabwe,-19.78333,29.36667,Q32017959
150550,131503,Shurugwi,1957,MI,Midlands Province,247,ZW,Zimbabwe,-19.67016,30.00589,Q32019023
150551,131504,Shurugwi District,1957,MI,Midlands Province,247,ZW,Zimbabwe,-19.75000,30.16667,Q7505444


In [230]:
# Map each city = name to country code through this csv
# Create mappings between cc and sub-region
mappings = {}
for city, count in cities_counts_dict.items():
    try:
        # TODO may have to change to region instead
        cc = cities_df[cities_df['name'] == city]['country_code'].values[0]
        mappings[city] = cc
    except:
        # hit end of series
        pass
print(mappings)
gen_df["gen_city_to_country_code"] = gen_df["city"].map(mappings)
gen_df = gen_df.drop(["city"],axis=1)
gen_df

{"'t Horntje": 'NL', 'Aabenraa': 'DK', 'Aach': 'DE', 'Aachen': 'DE', 'Aalsmeer': 'NL', 'Aalst': 'BE', 'Aarburg': 'CH', 'Aarhus': 'DK', 'Aartselaar': 'BE', 'Abakaliki': 'NG', 'Abakan': 'RU', 'Abancay': 'PE', 'Abano Terme': 'IT', 'Abarán': 'ES', 'Abasolo': 'MX', 'Abbekerk': 'NL', 'Abbeville': 'FR', 'Abbots Langley': 'GB', 'Abbotsford': 'AU', 'Abbottabad': 'PK', 'Abeokuta': 'NG', 'Aberdare': 'AU', 'Aberdeen': 'AU', 'Abernethy': 'GB', 'Aberystwyth': 'GB', 'Abha': 'SA', 'Abidjan': 'CI', 'Abiko': 'JP', 'Abilene': 'US', 'Abingdon': 'GB', 'Absam': 'AT', 'Absecon': 'US', 'Abuja': 'NG', 'Acacia Ridge': 'AU', 'Acacías': 'CO', 'Acate': 'IT', 'Accra': 'GH', 'Accrington': 'GB', 'Acerra': 'IT', 'Acheng': 'CN', 'Acton': 'AU', 'Actopan': 'MX', 'Acushnet': 'US', 'Acworth': 'US', 'Ada': 'US', 'Adamantina': 'BR', 'Adams': 'PH', 'Addis Ababa': 'ET', 'Addison': 'US', 'Addlestone': 'GB', 'Adeje': 'ES', 'Adelaide': 'AU', 'Adelsheim': 'DE', 'Adliswil': 'CH', 'Adrar': 'DZ', 'Adrian': 'RO', 'Aerzen': 'DE', 'Aesc

,course_id,user_id,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,gen_cc_by_ip_to_sub-region,gen_city_to_country_code
0,HarvardX/PH525.1x/1T2018,29940,78713,NaN,NaN,NaN,0,0,0,0,0,0,0,Northern America,US
1,HarvardX/PH525.1x/1T2018,37095,NaN,b,1991.0,m,0,0,0,0,0,0,0,Southern Asia,BD
2,HarvardX/PH525.1x/1T2018,45634,NaN,m,1982.0,m,0,0,0,0,0,0,0,Latin America and the Caribbean,CO
3,HarvardX/PH525.1x/1T2018,52234,NaN,p,1988.0,m,0,0,0,0,0,0,0,Northern Europe,NaN
4,HarvardX/PH525.1x/1T2018,52238,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,Latin America and the Caribbean,MX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,HarvardX/Hum3.1x/1T2016,15291085,2752,jhs,2002.0,NaN,0,0,0,0,0,0,0,Australia and New Zealand,AU
199995,HarvardX/Hum3.1x/1T2016,15292716,620000,NaN,NaN,NaN,0,0,0,0,0,0,0,Eastern Europe,RU
199996,HarvardX/Hum3.1x/1T2016,15295130,NaN,b,1996.0,f,0,0,0,0,0,0,0,Western Asia,NaN
199997,HarvardX/Hum3.1x/1T2016,15296396,02050,NaN,2000.0,NaN,1,0,0,0,1,0,0,Northern America,GB


In [232]:
# Validate each unique val in this col has at least 5 entries
gen_lvl_anon = level_k_anon(gen_df, ["gen_city_to_country_code"])
gen_lvl_anon # Anon is 1, this is not properly generalized!

1

In [235]:
# That wasn't enough generalization... go from country code to sub-region again
gen_cities_counts_dict = get_grouped_row_counts(gen_df, ["gen_city_to_country_code"]).to_dict()
print(cities_counts_dict)
# Map each cc_by_ip = alpha-2 to sub-region-code through this csv
# Create mappings between cc and sub-region
mappings = {}
for cc, count in gen_cities_counts_dict.items():
    try:
        # TODO may have to change to region instead
        sub_region = country_code_df[country_code_df['alpha-2'] == cc]['sub-region'].values[0]
        mappings[cc] = sub_region
    except:
        # hit end of series
        pass
print(mappings)
gen_df["gen_city_to_sub-region"] = gen_df["gen_city_to_country_code"].map(mappings)
# gen_df = gen_df.drop(["cc_by_ip"],axis=1)

{"'s-hertogenbosch": 3, "'t Horntje": 1, 'A Coruña': 21, 'Aabenraa': 1, 'Aach': 1, 'Aachen': 30, 'Aalsmeer': 4, 'Aalst': 5, 'Aarburg': 1, 'Aarhus': 36, 'Aartselaar': 1, 'Aas': 3, 'Abakaliki': 1, 'Abakan': 2, 'Abancay': 2, 'Abano Terme': 2, 'Abarán': 1, 'Abasolo': 1, 'Abbekerk': 1, 'Abbeville': 3, 'Abbots Langley': 2, 'Abbotsford': 16, 'Abbottabad': 7, 'Abdullah': 1, 'Abeokuta': 5, 'Aberdare': 4, 'Aberdeen': 49, 'Abernethy': 1, 'Aberystwyth': 4, 'Abha': 1, 'Abidjan': 65, 'Abiko': 4, 'Abilene': 14, 'Abingdon': 13, 'Abira': 1, 'Abrams': 1, 'Absam': 1, 'Absecon': 59, 'Abu Dhabi': 174, 'Abuja': 176, 'Abymes': 1, 'Acacia Ridge': 1, 'Acacías': 2, 'Acapulco': 10, 'Acate': 1, 'Accra': 438, 'Accrington': 2, 'Acerra': 1, 'Acheng': 1, 'Achrafieh': 1, 'Acme': 1, 'Acolman': 1, 'Acton': 30, 'Actopan': 2, 'Acushnet': 1, 'Acworth': 15, 'Ad Dammam': 1, 'Ada': 4, 'Adamantina': 1, 'Adams': 2, 'Adana': 27, 'Addis Ababa': 178, 'Addison': 14, 'Addlestone': 3, 'Adeje': 1, 'Adelaide': 160, 'Adelsheim': 1, 'Adl

In [238]:
gen_df = gen_df.drop(["gen_city_to_country_code"], axis=1)

In [239]:
gen_df

,course_id,user_id,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,gen_cc_by_ip_to_sub-region,gen_city_to_sub-region
0,HarvardX/PH525.1x/1T2018,29940,78713,NaN,NaN,NaN,0,0,0,0,0,0,0,Northern America,Northern America
1,HarvardX/PH525.1x/1T2018,37095,NaN,b,1991.0,m,0,0,0,0,0,0,0,Southern Asia,Southern Asia
2,HarvardX/PH525.1x/1T2018,45634,NaN,m,1982.0,m,0,0,0,0,0,0,0,Latin America and the Caribbean,Latin America and the Caribbean
3,HarvardX/PH525.1x/1T2018,52234,NaN,p,1988.0,m,0,0,0,0,0,0,0,Northern Europe,NaN
4,HarvardX/PH525.1x/1T2018,52238,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,Latin America and the Caribbean,Latin America and the Caribbean
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,HarvardX/Hum3.1x/1T2016,15291085,2752,jhs,2002.0,NaN,0,0,0,0,0,0,0,Australia and New Zealand,Australia and New Zealand
199995,HarvardX/Hum3.1x/1T2016,15292716,620000,NaN,NaN,NaN,0,0,0,0,0,0,0,Eastern Europe,Eastern Europe
199996,HarvardX/Hum3.1x/1T2016,15295130,NaN,b,1996.0,f,0,0,0,0,0,0,0,Western Asia,NaN
199997,HarvardX/Hum3.1x/1T2016,15296396,02050,NaN,2000.0,NaN,1,0,0,0,1,0,0,Northern America,Northern Europe


In [241]:
# Validate each unique val in this col has at least 5 entries
gen_lvl_anon = level_k_anon(gen_df, ["gen_city_to_sub-region"])
gen_lvl_anon # Anon is 2, this is not properly generalized!

2

In [243]:
# That wasn't enough generalization... go from sub-region to region
gen_cities2_counts_dict = get_grouped_row_counts(gen_df, ["gen_city_to_sub-region"]).to_dict()
print(gen_cities2_counts_dict)
# Map each sub-region to region through this csv
# Create mappings between sub-region and region
mappings = {}
for subregion, count in gen_cities2_counts_dict.items():
    try:
        # TODO may have to change to region instead
        region = country_code_df[country_code_df['sub-region'] == subregion]['region'].values[0]
        mappings[subregion] = region
    except:
        # hit end of series
        pass
print(mappings)
gen_df["gen_city_to_region"] = gen_df["gen_city_to_sub-region"].map(mappings)
# gen_df = gen_df.drop(["cc_by_ip"],axis=1)

{'Australia and New Zealand': 15200, 'Central Asia': 185, 'Eastern Asia': 5936, 'Eastern Europe': 4383, 'Latin America and the Caribbean': 20437, 'Melanesia': 54, 'Micronesia': 4, 'Northern Africa': 1186, 'Northern America': 40779, 'Northern Europe': 7283, 'Polynesia': 2, 'South-eastern Asia': 7937, 'Southern Asia': 13344, 'Southern Europe': 4893, 'Sub-Saharan Africa': 4264, 'Western Asia': 2525, 'Western Europe': 7172, nan: 64415}
{'Australia and New Zealand': 'Oceania', 'Central Asia': 'Asia', 'Eastern Asia': 'Asia', 'Eastern Europe': 'Europe', 'Latin America and the Caribbean': 'Americas', 'Melanesia': 'Oceania', 'Micronesia': 'Oceania', 'Northern Africa': 'Africa', 'Northern America': 'Americas', 'Northern Europe': 'Europe', 'Polynesia': 'Oceania', 'South-eastern Asia': 'Asia', 'Southern Asia': 'Asia', 'Southern Europe': 'Europe', 'Sub-Saharan Africa': 'Africa', 'Western Asia': 'Asia', 'Western Europe': 'Europe'}


In [244]:
gen_df

,course_id,user_id,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,gen_cc_by_ip_to_sub-region,gen_city_to_sub-region,gen_city_to_region
0,HarvardX/PH525.1x/1T2018,29940,78713,NaN,NaN,NaN,0,0,0,0,0,0,0,Northern America,Northern America,Americas
1,HarvardX/PH525.1x/1T2018,37095,NaN,b,1991.0,m,0,0,0,0,0,0,0,Southern Asia,Southern Asia,Asia
2,HarvardX/PH525.1x/1T2018,45634,NaN,m,1982.0,m,0,0,0,0,0,0,0,Latin America and the Caribbean,Latin America and the Caribbean,Americas
3,HarvardX/PH525.1x/1T2018,52234,NaN,p,1988.0,m,0,0,0,0,0,0,0,Northern Europe,NaN,NaN
4,HarvardX/PH525.1x/1T2018,52238,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,Latin America and the Caribbean,Latin America and the Caribbean,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,HarvardX/Hum3.1x/1T2016,15291085,2752,jhs,2002.0,NaN,0,0,0,0,0,0,0,Australia and New Zealand,Australia and New Zealand,Oceania
199995,HarvardX/Hum3.1x/1T2016,15292716,620000,NaN,NaN,NaN,0,0,0,0,0,0,0,Eastern Europe,Eastern Europe,Europe
199996,HarvardX/Hum3.1x/1T2016,15295130,NaN,b,1996.0,f,0,0,0,0,0,0,0,Western Asia,NaN,NaN
199997,HarvardX/Hum3.1x/1T2016,15296396,02050,NaN,2000.0,NaN,1,0,0,0,1,0,0,Northern America,Northern Europe,Europe


In [245]:
gen_df = gen_df.drop(["gen_city_to_sub-region"],axis=1)
gen_df

,course_id,user_id,postalCode,LoE,YoB,gender,nforum_posts,nforum_votes,nforum_endorsed,nforum_threads,nforum_comments,nforum_pinned,nforum_events,gen_cc_by_ip_to_sub-region,gen_city_to_region
0,HarvardX/PH525.1x/1T2018,29940,78713,NaN,NaN,NaN,0,0,0,0,0,0,0,Northern America,Americas
1,HarvardX/PH525.1x/1T2018,37095,NaN,b,1991.0,m,0,0,0,0,0,0,0,Southern Asia,Asia
2,HarvardX/PH525.1x/1T2018,45634,NaN,m,1982.0,m,0,0,0,0,0,0,0,Latin America and the Caribbean,Americas
3,HarvardX/PH525.1x/1T2018,52234,NaN,p,1988.0,m,0,0,0,0,0,0,0,Northern Europe,NaN
4,HarvardX/PH525.1x/1T2018,52238,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,Latin America and the Caribbean,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,HarvardX/Hum3.1x/1T2016,15291085,2752,jhs,2002.0,NaN,0,0,0,0,0,0,0,Australia and New Zealand,Oceania
199995,HarvardX/Hum3.1x/1T2016,15292716,620000,NaN,NaN,NaN,0,0,0,0,0,0,0,Eastern Europe,Europe
199996,HarvardX/Hum3.1x/1T2016,15295130,NaN,b,1996.0,f,0,0,0,0,0,0,0,Western Asia,NaN
199997,HarvardX/Hum3.1x/1T2016,15296396,02050,NaN,2000.0,NaN,1,0,0,0,1,0,0,Northern America,Europe


In [247]:
# Validate each unique val in this col has at least 5 entries
gen_lvl_anon = level_k_anon(gen_df, ["gen_city_to_region"])
gen_lvl_anon # Anon is 5450, this is properly generalized!

5450

4.2 Step 2: check that generalization resulted in 5-anonymous dataframe

In [387]:
# gen_df = gen_df.drop(["gen_YoB_to_4_quantiles"], axis=1)

In [397]:
gen_df

,LoE,gender,gen_cc_by_ip_to_sub-region,gen_city_to_region,gen_postalCode_to_1st_digit,gen_YoB_to_10_quantiles,gen_nforum_posts_to_5_quantiles,gen_nforum_votes_to_4_quantiles,gen_nforum_endorsed_to_2_quantiles,gen_nforum_threads_to_2_quantiles,gen_nforum_comments_to_2_quantiles,gen_nforum_pinned,gen_nforum_events
0,NaN,NaN,Northern America,Americas,7,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
1,b,m,Southern Asia,Asia,NAN,"(1989.0, 1991.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
2,m,m,Latin America and the Caribbean,Americas,NAN,"(1980.0, 1984.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
3,p,m,Northern Europe,NaN,NAN,"(1987.0, 1989.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
4,NaN,NaN,Latin America and the Caribbean,Americas,NAN,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,jhs,NaN,Australia and New Zealand,Oceania,2,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199995,NaN,NaN,Eastern Europe,Europe,6,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199996,b,f,Western Asia,NaN,NAN,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199997,NaN,NaN,Northern America,Europe,0,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"


In [403]:
print(level_k_anon_num)

0


In [406]:
gen_df_mod

,LoE,gender,gen_city_to_region,gen_YoB_to_10_quantiles,gen_nforum_posts_to_5_quantiles,gen_nforum_votes_to_4_quantiles,gen_nforum_endorsed_to_2_quantiles,gen_nforum_threads_to_2_quantiles,gen_nforum_comments_to_2_quantiles,gen_nforum_pinned,gen_nforum_events
0,NaN,NaN,Americas,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
1,b,m,Asia,"(1989.0, 1991.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
2,m,m,Americas,"(1980.0, 1984.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
3,p,m,NaN,"(1987.0, 1989.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
4,NaN,NaN,Americas,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
...,...,...,...,...,...,...,...,...,...,...,...
199994,jhs,NaN,Oceania,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199995,NaN,NaN,Europe,NaN,"(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199996,b,f,NaN,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"
199997,NaN,NaN,Europe,"(1995.0, 2018.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]","(-0.001, 1.0]"


In [414]:
gen_quasi_ids = ['gen_LoE', 'gender', 'gen_city_to_region', 'gen_YoB_to_10_quantiles', 'gen_nforum_posts_to_5_quantiles', 'gen_nforum_votes_to_4_quantiles', 'gen_nforum_endorsed_to_2_quantiles', 'gen_nforum_threads_to_2_quantiles', 'gen_nforum_comments_to_2_quantiles', 'gen_nforum_pinned', 'gen_nforum_events']
# Group by set of quasi id values
min_rows = None
quasi_id_grouped_df = gen_df_mod.groupby(gen_quasi_ids, dropna=False)
for name, group in quasi_id_grouped_df:
    print(group.shape[0])
    if (min_rows is None or group.shape[0] < min_rows):
        min_rows = group.shape[0]
# # Get number of rows in each gruop
# grouped_row_counts = quasi_id_grouped_df.size()
# print(grouped_row_counts)
# # rouped_row_counts2 = quasi_id_grouped_df.size()
# # Min number of rows in a group = level of k-anonymity
# level_k_anon_num = min(grouped_row_counts)

21
1
1
27
1
2
1
1
48
1
1
97
2
3
1
1
1
111
1
4
1
1
2
1
1
1
99
1
1
1
76
2
1
1
1
71
1
1
1
31
16
1
1201
12
5
9
5
1
1
2
6
5
1
1
1
1
1
5
1
2
1
3
3
2
2
2
2
880
1
9
3
6
5
1
3
4
1
2
1
1
1
736
8
1
4
5
1
1
2
2
1
2
2
1
2
3
1
915
2
1
7
5
1
1
3
1
2
1
1
2
2
1
1
1055
2
1
6
6
1
3
2
1
1
1
1
1
1
1019
5
2
8
1
1
1
2
1
1
1
1
1
2
1147
2
4
2
1
1
4
1
2
1
2
1
797
3
2
4
1
2
1
1
1
1
377
1
1
2
1
1
227
1
1
94
1
1
1
1
1
174
1
2
1
1
1
1
177
1
2
1
1
1
271
2
1
1
1
1
415
3
1
1
2
2
2
1
1
452
2
3
2
2
1
1
1
688
3
2
1
1
1
1
1
1
632
3
4
2
2
1
1
1
364
1
1
1
155
1
1
229
2
2
1
1
1
1
1
228
1
2
2
4
2
1
1
1
1
205
1
286
1
3
1
1
1
1
1
338
2
1
1
1
1
323
2
1
2
1
2
1
1
2
425
1
1
1
1
2
1
1
288
1
1
1
1
124
1
3
1
54
283
5
2
1
3
1
2
1
1
2
1
1
1
1
1
2
232
1
5
2
2
1
191
1
1
3
1
265
1
1
2
1
1
1
298
1
5
2
2
1
332
1
2
4
1
1
309
1
1
2
1
1
1
1
192
1
1
1
2
79
1
29
668
1
3
1
5
2
2
2
1
2
1
1
798
4
1
1
4
2
1
1
1
1
785
1
1
1
1
7
1
3
1
1
1
1
3
915
1
3
1
4
1
1
1
1
1
1
1
1
1
1
1116
3
2
3
2
1
1
1
1
1
1
2
1
1027
2
1
3
2
1
3
1
1
1
1
1
1
1
1362
4
1
1
1
6
2
2

In [415]:
min_rows

1

**5. See if you can use some combination of these mechanisms to produce a 5-anonymous data set**
Balancing Generalization by Concentration, House, and Year to maintain the most descriptive categories while having 5-anonymity 